# Motivation

The dataset employed in this project is the 'Air Traffic Passenger Statistics' of the San Francisco Airport. The dataset holds information about the Monthly Passenger Traffic Statistics. In order words it records all flights departing, arriving, or stopping over in San Francisco over a span of 23 years. Each flight is logged once per month, and the passenger count is calculated by aggregating all passengers on that flight during the respective month.
We opted for this specific dataset due to its expansive scope, encompassing 15 attributes and spanning 23 years, which offers ample opportunities for diverse studies related to San Francisco Airport and its air traffic. Moverover we personally think that the topic could be wordwide interesting given that airports serve as critical infrastructure facilitating thousands of flights daily, orchestrated through meticulous scheduling.

### What was your goal for the end user's experience?

# Basic stats

The dataset itself contains data from july 1999 to january 2024. Since Airport data is seasonal in nature, any comparative analyses should be done on a period-over-period basis (i.e. January 2010 vs. January 2009) as opposed to period-to-period (i.e. January 2010 vs. February 2010). In light of these, the dataset was adjusted to include only complete years.

In [ ]:
df = pd.read_csv('C:/Users/Paula/Documents/DTU/Spring_2024/Social_data/project/FinalProject---SocialData24.github.io/Air_Traffic_Passenger_Statistics_20240408.csv')

In [ ]:
# Delete duplicates
df = df.drop_duplicates()

# Find rows with one or more empty columns
empty_rows = df[df.isnull().any(axis=1)]

# Find columns with empty values
empty_columns = []
for index, row in empty_rows.iterrows():
    empty_columns.extend([col for col in df.columns if pd.isnull(row[col])])

unique_empty_columns = list(set(empty_columns))
print(unique_empty_columns)

Since the only columns that have empty values are these two, we decided to remove them because the information in them is the same as the one in "Operating Airline" and "Published Airline" columns.

In [ ]:
df.drop(columns=['Published Airline IATA Code', 'Operating Airline IATA Code'], inplace=True)

# Convert columns to datetime format
df['Activity Period Start Date'] = pd.to_datetime(df['Activity Period Start Date'])
df.drop(columns=['Activity Period'], inplace=True)

# Clean unnecessary columns
df.drop(columns=['data_as_of', 'data_loaded_at'], inplace=True)

In [ ]:
# Delete the first and last year of the dataset since they are not complete
df = df[(df['Activity Period Start Date'].dt.year != 1999) & (df['Activity Period Start Date'].dt.year != 2024)]

In [ ]:
df.head()
df.describe()

Total amount of passengers without taking into account what Activity Type are they

In [ ]:
import matplotlib.pyplot as plt

# Group by year and sum 'Passenger Count'
yearly_passenger_count = df.groupby(df['Activity Period Start Date'].dt.year)['Passenger Count'].sum().reset_index()

# Plot the summed 'Passenger Count' for each year
plt.figure(figsize=(10, 6))
plt.plot(yearly_passenger_count['Activity Period Start Date'], yearly_passenger_count['Passenger Count'], marker='o')
plt.title('Total Passenger Count Over Time')
plt.xlabel('Year')
plt.ylabel('Total Passenger Count')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()